In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Inspecting Case Data

## CasesWidget

When working in a Jupyter notebook, you can quickly get a plot of the data you have recorded from your analysis using a `CasesWidget`.  See the following example.

In [2]:
import openmdao.api as om

recorder = om.SqliteRecorder("cases.sql")

prob = om.Problem()

prob.model.add_subsystem('paraboloid', om.ExecComp('f = (x-3)**2 + x*y + (y+4)**2 - 3'))

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'

prob.model.add_design_var('paraboloid.x', lower=-50, upper=50)
prob.model.add_design_var('paraboloid.y', lower=-50, upper=50)
prob.model.add_objective('paraboloid.f')

prob.driver.add_recorder(recorder)
prob.add_recorder(recorder)

prob.driver.recording_options['record_objectives'] = True
prob.driver.recording_options['record_constraints'] = True
prob.driver.recording_options['record_desvars'] = True

prob.setup()

prob.set_val('paraboloid.x', 3.0)
prob.set_val('paraboloid.y', -4.0)

prob.run_model()
prob.record("initial")
prob.run_driver()
prob.record("final")

Model viewer data has already has already been recorded for Driver.
Optimization terminated successfully    (Exit mode 0)
            Current function value: [-27.33333333]
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


/home/swryan/dev/OpenMDAO/openmdao/recorders/sqlite_recorder.py:223: UserWarning:The existing case recorder file, cases.sql, is being overwritten.


Having recorded our driver iterations and final problem state into the file "cases.sql", we can now instantiate a `CasesWidget` to quickly and easily generate plots of our output variables across iterations.

Using the `Source` selector we can choose to inspect either the cases recorded by the driver or the cases recorded by the problem.  Having chosen the source, we can then select the range of `Cases` to inspect and then finally the `Output` variable to plot.

In [4]:
om.CasesWidget("cases.sql");

interactive(children=(Dropdown(description='Source:', options=('driver', 'problem'), value='driver'), IntRange…